# Project 3 - Part 2a Extract from TMDB (Core)
<b> Your Stakeholder Wants More Data!
    
- After investigating the preview of your data from Part 1, your stakeholder realized that there is no financial information included in the IMDB data (e.g. budget or revenue).
    - This will be a major roadblock when attempting to analyze which movies are successful and must be addressed before you will be able to determine which movies are successful.
- Your stakeholder identified The Movie Database (TMDB) as a great source of financial data (https://www.themoviedb.org/). Thankfully, TMDB offers a free API for programmatic access to their data!
- Your stakeholder wants you to extract the budget, revenue, and MPAA Rating (G/PG/PG-13/R), which is also called "Certification".
- Note: this process can take a long time and may need to run overnight.


In [1]:
# imports
import json, os, time
import tmdbsimple as tmdb
import pandas as pd
from tqdm.notebook import tqdm_notebook

In [2]:
# load API 
with open('/Users/j29ma/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
# Display the keys of the loaded dict
login.keys()

dict_keys(['api-key'])

In [3]:
# API key
tmdb.API_KEY =  login['api-key']

In [4]:
# folder for API call data
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok = True)
os.listdir(FOLDER)

['final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json',
 'tmdb_results_combined.csv.gz']

In [5]:
# make a movie object using the .Movies function from tmdb
movie = tmdb.Movies(603)

In [6]:
# movie objects have a .info dictionary 
info = movie.info()
info

{'adult': False,
 'backdrop_path': '/oMsxZEvz9a708d49b6UdZK1KAo5.jpg',
 'belongs_to_collection': {'id': 2344,
  'name': 'The Matrix Collection',
  'poster_path': '/bV9qTVHTVf0gkW0j7p7M0ILD4pG.jpg',
  'backdrop_path': '/bRm2DEgUiYciDw3myHuYFInD7la.jpg'},
 'budget': 63000000,
 'genres': [{'id': 28, 'name': 'Action'},
  {'id': 878, 'name': 'Science Fiction'}],
 'homepage': 'http://www.warnerbros.com/matrix',
 'id': 603,
 'imdb_id': 'tt0133093',
 'original_language': 'en',
 'original_title': 'The Matrix',
 'overview': 'Set in the 22nd century, The Matrix tells the story of a computer hacker who joins a group of underground insurgents fighting the vast and powerful computers who now rule the earth.',
 'popularity': 69.872,
 'poster_path': '/aOIuZAjPaRIE6CMzbazvcHuHXDc.jpg',
 'production_companies': [{'id': 79,
   'logo_path': '/tpFpsqbleCzEE2p5EgvUq6ozfCA.png',
   'name': 'Village Roadshow Pictures',
   'origin_country': 'US'},
  {'id': 372,
   'logo_path': None,
   'name': 'Groucho II Film

## Query Movies by ID
### IMDB ID

In [7]:
info['imdb_id']

'tt0133093'

### Budget

In [8]:
info['budget']

63000000

#### Revenue

In [9]:
info['revenue']

463517383

### Certification

In [10]:
# example from package README
# source = https://github.com/celiao/tmdbsimpleresponse = movie.releases()
releases = movie.releases()
for c in movie.countries:
    if c['iso_3166_1'] == 'US':
        print(c['certification'])

R
R


# <b> Specifications - Financial Data
    
- Your stakeholder would like you to extract and save the results for movies that meet all of the criteria established in part 1 of the project (You should already have a filtered dataframe saved from part one as a csv.gz file)

- As a proof-of-concept, they requested you perform a test extraction of movies that started in 2000 or 2001

- Each year should be saved as a separate .csv.gz file

Hint: Use the two custom functions from the lessons (Intro to TMDB API, and Efficient TMDB API Calls). Be sure to define these functions prior to calling them in your code!

- One function will add the certification (MPGG Rating) to movie.info
- The other function will help you append/extend a JSON file with Python
    
<b> Confirm Your API Function works.

In order to ensure your function for extracting movie data from TMDB is working, test your function on these 2 movie ids: tt0848228 ("The Avengers") and tt0332280 ("The Notebook"). Make sure that your function runs without error and that it returns the correct movie's data for both test ids.

Hint: Ideally you can organize the code segments from the previous lesson to create an outer and inner loop, but if you get stuck, you can complete 1 year at a time.

- Once you have retrieved and saved the final results to 2 separate .csv.gz files, move on to a new Exploratory Data Analysis notebook to explore the following questions.

## Certification (MPGG Rating) Function

In [11]:
# function to add certification to movie.info dict
def get_movie_with_rating(movie_id):
    """Adapted from source = https://github.com/celiao/tmdbsimple"""
    
    # set movie id
    movie = tmdb.Movies(movie_id)

    # save the .info and .releases dictionaries
    info = movie.info()
    releases = movie.releases()
    
    # only get releases of the movie in the US
    for c in releases['countries']:
        
        # if country abbreviation == US
        if c['iso_3166_1'] == 'US':
            
            # save certification key into info dict
            info['certification'] = c['certification']
            
    return info

In [12]:
def information(movie):
    print(f'Budget: $',movie['budget'])
    print(f'Revenue: $',movie['revenue'])
    print(f'Certification: ',movie['certification'])

### 1st Test

In [13]:
movie_id1 = 'tt0848228'
info1 = get_movie_with_rating(movie_id1)

In [14]:
information(info1)

Budget: $ 220000000
Revenue: $ 1518815515
Certification:  PG-13


### 2nd Test

In [15]:
movie_id2 = 'tt0332280'
info2 = get_movie_with_rating(movie_id2)

In [16]:
information(info2)

Budget: $ 29000000
Revenue: $ 115603229
Certification:  PG-13


### 3rd Test

In [17]:
# testing our function by looping through a list of ids
test_ids = ["tt0848228", "tt0115937","tt0848228","tt0332280"]
results = []
for movie_id in test_ids:
    
    try:
        movie_info = get_movie_with_rating(movie_id)
        results.append(movie_info)
        
    except: 
        pass
    
pd.DataFrame(results)

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,"{'id': 86311, 'name': 'The Avengers Collection...",220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",https://www.marvel.com/movies/the-avengers,24428,tt0848228,en,The Avengers,...,1518815515,143,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Some assembly required.,The Avengers,False,7.708,28737,PG-13
1,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,"{'id': 86311, 'name': 'The Avengers Collection...",220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",https://www.marvel.com/movies/the-avengers,24428,tt0848228,en,The Avengers,...,1518815515,143,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Some assembly required.,The Avengers,False,7.708,28737,PG-13
2,False,/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg,None,29000000,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",http://www.newline.com/properties/notebookthe....,11036,tt0332280,en,The Notebook,...,115603229,123,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Behind every great love is a great story.,The Notebook,False,7.880,10381,PG-13


## Extend JSON file Function

In [18]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

## Loading Filtered Basics Data

In [19]:
basics = pd.read_csv('Data/title_basics.csv.gz')
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0043139,movie,Life of a Beijing Policeman,Wo zhe yi bei zi,0,2013,NaN,120,"Drama,History"
2,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
3,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
...,...,...,...,...,...,...,...,...,...
86751,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019,NaN,74,Drama
86752,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019,NaN,97,"Comedy,Drama,Fantasy"
86753,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,NaN,51,Drama
86754,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,95,"Action,Adventure,Thriller"


In [20]:
# create list of years to extract from API
YEARS_TO_GET = [2000, 2001]

In [21]:
# create empty list to save errors
errors = []

In [22]:
# set up progress bar for outer loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc = 'YEARS', position = 0):
    
    # select JSON_FILE name to save results in progress
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    
    # check if file already exists
    file_exists = os.path.isfile(JSON_FILE)
    if file_exists == False:
        
        # save an empty dict with "imdb_id" = 0 to new file
        with open(JSON_FILE, 'w') as f:
            json.dump([{'imdb_id': 0}], f)
    
    # define df for current year
    df = basics.loc[basics['startYear'] == YEAR].copy()
    
    # save movie ids from df['tconst'] to list
    movie_ids = df['tconst'].copy()
    
    # load any existing data from json into previous_df
    previous_df = pd.read_json(JSON_FILE)
    
    # filter out any movies that already exist in JSON_FILE
    # (so as not to repeat any API calls for the same movie)
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    
    # inner loop to make API calls
    # iterate through list of movie_ids_to_get
    # some movies with imdb_ids don't exist in tmdb, so use
    # try/except to work through errors
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                 desc = f'Movies from {YEAR}',
                                 position = 1,
                                 leave = True):
        
        try:
            # retrieve data for movie_id
            temp = get_movie_with_rating(movie_id)
            
            # append/extend results to JSON_FILE
            write_json(temp, JSON_FILE)
            
            # pause to prevent overwhelming server with calls
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])
            
    # save year's results in csv.gz file
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz",
                        compression = "gzip",
                        index = False)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/210 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/234 [00:00<?, ?it/s]

In [23]:
print(f"- Total errors: {len(errors)}")

- Total errors: 442
